In [63]:
import plyvel
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
import re
from io import BytesIO
import shutil
from PIL import Image
import os
import json
import unicodedata

In [10]:
# Adjust to your local path:
path = '/Users/douwe/Downloads/Icons8 v5.6.3/icons'
db = plyvel.DB(path)

IOError: b'IO error: lock /Users/douwe/Downloads/Icons8 v5.6.3/icons/LOCK: already held by process'

'FULL STOP'

In [68]:
splitter = re.compile(b'[\x00-\x08]')

def parse_value(value):
    res = {}
    prev = ''
    for elem in splitter.split(value):
        if not elem:
            continue
        try:
            elem = elem.decode('utf8')
        except UnicodeDecodeError:
            continue
        if elem in ('category', 'name', 'platform', 'canonical_name', 'svg'):
            if elem == 'name' and len(prev) == 1:
                prev = 'u_' + unicodedata.name(prev).lower().replace(' ', '_')
            res[elem] = prev
        prev = elem
    return res

for _, value in db:
    res = parse_value(value)
    break
res

{'canonical_name': 'Address Book',
 'category': 'Messaging',
 'name': 'address_book',
 'platform': 'windows',
 'svg': '<?xml version="1.0" encoding="utf-8"?>\n<!-- Generator: Adobe Illustrator 15.0.0, SVG Export Plug-In . SVG Version: 6.00 Build 0)  -->\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" id="Layer_1" x="0px" y="0px" width="26px" height="26px" viewBox="0 0 26 26" enable-background="new 0 0 26 26" xml:space="preserve">\n\n<path fill="#000000" d="M12.792,9.803c-0.221,0-0.423,0.053-0.618,0.166c-0.2,0.111-0.399,0.302-0.594,0.562  c-0.198,0.266-0.359,0.593-0.48,0.977c-0.121,0.385-0.184,0.741-0.184,1.061c0,0.488,0.115,0.867,0.34,1.126  c0.371,0.428,0.811,0.485,1.343,0.205c0.225-0.121,0.444-0.304,0.653-0.543c0.207-0.241,0.382-0.553,0.517-0.929  c0.136-0.378,0.206-0.759,0.206-1.137c0-0.481-0.119-0.855-0.354-1.112C13.393,9.926,13.12,9.

In [70]:
icons = {}

for key, value in db:
    try:
        res = parse_value(value)
    except ValueError:
        continue
    if res.get('platform') == 'ios':
        name = res.get('name')
        if not name:
            name = res.get('canonical_name')
            if not name:
                continue
            name = name.lower().replace(' ', '_')
        icons[name] = res
len(icons)

9942

In [71]:
SIZES = (16, 28, 32, 50)

if os.path.isdir('icons'):
    shutil.rmtree('icons')
os.mkdir('icons')
for size in SIZES:
    os.mkdir('icons/png%s' % size)
os.mkdir('icons/svg')

In [72]:
saved = []
for icon in icons.values():
    icon = dict(icon)
    if not 'svg' in icon:
        continue
    svg = icon.pop('svg')
    try:
        drawing = svg2rlg(BytesIO(svg.encode('utf8')))
    except ValueError:
        continue
    except AttributeError:
        continue
    open('icons/svg/%s.svg' % icon['name'], 'w').write(svg)
    p = renderPM.drawToPIL(drawing)
    for size in SIZES:
        resized = p.resize((size, size), Image.ANTIALIAS)
        resized.save('icons/png%s/%s.png' % (size, icon['name']))
    saved.append(icon)
json.dump(saved, open('icons/index.json', 'w'), indent=2)
len(saved)
    

Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.
Adding box instead of image.


9900

In [67]:
icon['name']

'/'